# Same Keywords Long Distance Analysis

This notebook analyzes news data with the same keywords to find pairs with the lowest similarity scores.

## Step 1: Import Required Libraries

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from datetime import datetime
from itertools import combinations
import os
from typing import List, Union
import warnings
warnings.filterwarnings('ignore')

## Step 2: Read Database Path from .env File

In [ ]:
def read_db_path_from_env(env_file: str = '.env') -> str:
    """
    Read database path from .env file.
    Expected format: db_path: xxx
    
    Args:
        env_file: Path to the .env file
        
    Returns:
        Database path as string
    """
    try:
        with open(env_file, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line.startswith('db_path:'):
                    db_path = line.split('db_path:')[1].strip()
                    return db_path
        raise ValueError("db_path not found in .env file")
    except FileNotFoundError:
        raise FileNotFoundError(f"{env_file} file not found")

# Read database path
db_path = read_db_path_from_env()
print(f"Database path: {db_path}")

## Step 3: Query Database and Create Initial Dataframe

In [ ]:
def query_news_data(db_path: str) -> pd.DataFrame:
    """
    Query the database to get news data with keywords.
    Joins main_news_data with serpapi_data to get the query field.
    
    Args:
        db_path: Path to the SQLite database
        
    Returns:
        DataFrame with id, serpapi_id, news, date, and keywords columns
    """
    try:
        conn = sqlite3.connect(db_path)
        
        query = """
        SELECT 
            m.id,
            m.serpapi_id,
            m.news,
            m.date,
            s.query as keywords
        FROM main_news_data m
        JOIN serpapi_data s ON m.serpapi_id = s.id
        WHERE m.news IS NOT NULL AND m.news != ''
        ORDER BY s.query, m.date
        """
        
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        # Convert date to datetime
        df['date'] = pd.to_datetime(df['date'])
        
        print(f"Total records loaded: {len(df)}")
        print(f"Unique keywords: {df['keywords'].nunique()}")
        
        return df
    
    except Exception as e:
        raise Exception(f"Error querying database: {str(e)}")

# Load data
df = query_news_data(db_path)
df.head()

## Step 4: Define Modular Embedding Function

In [ ]:
# Initialize the embedding model (global variable for efficiency)
embedding_model = None

def get_embeddings(texts: Union[str, List[str]], model_name: str = "Qwen/Qwen3-Embedding-0.6B") -> np.ndarray:
    """
    Generate embeddings for input text(s) using the specified model.
    This function is modular and can be easily replaced with different embedding models.
    
    Args:
        texts: Single text string or list of text strings to embed
        model_name: Name of the sentence transformer model to use
        
    Returns:
        numpy array of embeddings
    """
    global embedding_model
    
    # Load model only once
    if embedding_model is None:
        print(f"Loading embedding model: {model_name}...")
        embedding_model = SentenceTransformer(model_name)
        print("Model loaded successfully!")
    
    # Convert single string to list
    if isinstance(texts, str):
        texts = [texts]
    
    # Generate embeddings
    embeddings = embedding_model.encode(texts, show_progress_bar=True)
    
    return embeddings

def calculate_cosine_similarity(embedding1: np.ndarray, embedding2: np.ndarray) -> float:
    """
    Calculate cosine similarity between two embeddings.
    
    Args:
        embedding1: First embedding vector
        embedding2: Second embedding vector
        
    Returns:
        Cosine similarity score
    """
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

# Test the embedding function with a sample
print("Testing embedding function...")
test_embedding = get_embeddings("This is a test sentence.")
print(f"Embedding shape: {test_embedding.shape}")

## Step 5: Find Lowest Similarity Pairs for Each Keyword

In [ ]:
def find_lowest_similarity_pairs(df: pd.DataFrame) -> pd.DataFrame:
    """
    For each keyword with multiple records, find the pair with the lowest similarity.
    The pair consists of later news (news1) and earlier news (news2) based on date.
    
    Args:
        df: DataFrame with news data
        
    Returns:
        DataFrame with lowest similarity pairs
    """
    results = []
    
    # Group by keywords
    grouped = df.groupby('keywords')

    # Filter out groups with only one record
    df_filtered = df[df.groupby('keywords')['keywords'].transform('size') > 1]
    grouped = df_filtered.groupby('keywords')

    print(f"Processing {len(grouped)} keyword groups...")

    
    for keyword, group in grouped:
        # Skip keywords with only one record
        if len(group) < 2:
            continue
        
        # Sort by date to ensure proper ordering
        group = group.sort_values('date').reset_index(drop=True)
        
        # Get embeddings for all news in this group
        news_texts = group['news'].tolist()
        embeddings = get_embeddings(news_texts)
        
        # Find the pair with lowest similarity
        # Only consider pairs where news1 is later than news2
        min_similarity = float('inf')
        best_pair = None
        
        for i in range(len(group)):
            for j in range(i):
                # i is later (news1), j is earlier (news2)
                similarity = calculate_cosine_similarity(embeddings[i], embeddings[j])
                
                if similarity < min_similarity:
                    min_similarity = similarity
                    best_pair = (i, j)
        
        if best_pair is not None:
            idx1, idx2 = best_pair
            row1 = group.iloc[idx1]
            row2 = group.iloc[idx2]
            
            # Calculate date difference
            date_diff = (row1['date'] - row2['date']).days
            
            results.append({
                'id': row1['id'],
                'keywords': keyword,
                'news1': row1['news'],
                'news2': row2['news'],
                'similarity': min_similarity,
                'serpapi_id': row1['serpapi_id'],
                'date_diff': date_diff,
                'date1': row1['date'],
                'date2': row2['date']
            })
    
    # Create DataFrame from results
    result_df = pd.DataFrame(results)
    
    # Sort by similarity (lowest first)
    result_df = result_df.sort_values('similarity').reset_index(drop=True)
    
    print(f"Found {len(result_df)} keyword pairs with lowest similarity")
    
    return result_df

# Find lowest similarity pairs
result_df = find_lowest_similarity_pairs(df)
result_df.head(10)

## Step 6: Display Summary Statistics

In [ ]:
print("\n=== Summary Statistics ===")
print(f"Total pairs found: {len(result_df)}")
print(f"\nSimilarity score statistics:")
print(result_df['similarity'].describe())
print(f"\nDate difference statistics (days):")
print(result_df['date_diff'].describe())
print(f"\nLowest similarity score: {result_df['similarity'].min():.4f}")
print(f"Highest similarity score: {result_df['similarity'].max():.4f}")

## Step 7: Save Results to CSV

In [ ]:
def save_to_csv(df: pd.DataFrame, filename: str = 'same_keywords_lowest_similarity.csv'):
    """
    Save the DataFrame to a CSV file with UTF-8 with BOM encoding.
    
    Args:
        df: DataFrame to save
        filename: Output filename
    """
    try:
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"\nResults saved to: {filename}")
        print(f"Total rows saved: {len(df)}")
    except Exception as e:
        print(f"Error saving CSV: {str(e)}")

# Save results
save_to_csv(result_df)

## Step 8: Display Sample Results

In [ ]:
print("\n=== Sample Results (Top 5 Lowest Similarity Pairs) ===")
for idx, row in result_df.head(5).iterrows():
    print(f"\n--- Pair {idx + 1} ---")
    print(f"Keywords: {row['keywords']}")
    print(f"Similarity: {row['similarity']:.4f}")
    print(f"Date difference: {row['date_diff']} days")
    print(f"Date1: {row['date1']}")
    print(f"Date2: {row['date2']}")
    print(f"News1 (later): {row['news1'][:200]}...")
    print(f"News2 (earlier): {row['news2'][:200]}...")